In [1]:
import json
from transformers import AutoTokenizer,AutoModel
from torchmetrics import Accuracy
from torch import nn
from torch.utils.data import DataLoader
from model import BertBilstm
import torch
from utils import evaluate_recall,evaluate_f1,evaluate_precision,train_model,generate_collate_fn
from torchmetrics import Precision, Recall, F1Score

c:\Users\18528\anaconda3\envs\openai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('./data/train.json','r') as f:
    train_data=json.load(f)
with open('./data/test.json','r') as f:
    test_data=json.load(f)
with open('./data/label2index.json','r') as f:
    label2index=json.load(f)

precision = Precision(task="multiclass",num_classes=9)
recall = Recall(task="multiclass",num_classes=9)
f1 = F1Score(task="multiclass",num_classes=9)
device="cuda" if torch.cuda.is_available() else "cpu"
precision.to(device)
recall.to(device)
f1.to(device)
bertbilstm=BertBilstm().to(device)
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")
# net = AutoModel.from_pretrained("bert-base-chinese")
# net.to(device)
optimizer= torch.optim.Adam(bertbilstm.parameters(),lr = 1e-5)
loss_fn = nn.CrossEntropyLoss()
metrics_dict = {"precision":precision,
                "recall":precision,
                "f1":precision}

collate_fn=generate_collate_fn(tokenizer,label2index,device=device)
dl_train=DataLoader(train_data[:3000],batch_size=10,collate_fn=collate_fn)
dl_test=DataLoader(test_data[:1000],batch_size=10,collate_fn=collate_fn)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
"""
for name,params in net.named_parameters():
    params.requires_grad=False
"""

'\nfor name,params in net.named_parameters():\n    params.requires_grad=False\n'

In [5]:
dfhistory = train_model(bertbilstm,
    optimizer,
    loss_fn,
    metrics_dict,
    train_data = dl_train,
    val_data= dl_train,
    epochs=1,
    patience=5,
    monitor="val_f1", 
    mode="max"
    )


================================================================================2023-06-19 20:53:32
Epoch 1 / 1

  1%|▏         | 4/300 [00:12<13:59,  2.84s/it, train_f1=0.0561, train_loss=2.21, train_precision=0.0561, train_recall=0.0561]